<a href="https://colab.research.google.com/github/FatemeZamanian/DeepLearning/blob/main/facesRecognition_fs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,AveragePooling2D,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import cv2
img=cv2.imread('/content/drive/MyDrive/7-7 dataset/Ali_Khamenei/Ali-Khamenei-13_01.jpg')
print(img.shape)

(1024, 1024, 3)


In [3]:
batch_size=32
width=height=256

In [4]:
path='/content/drive/MyDrive/7-7 dataset'

data_generator=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data=data_generator.flow_from_directory(
    path,
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    subset='training',
)

val_data=data_generator.flow_from_directory(
    path,
    target_size=(width,height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    subset='validation',
)




Found 1091 images belonging to 14 classes.
Found 268 images belonging to 14 classes.


In [12]:
class MyModel(tf.keras.Model):
  def __init__(self,number_of_classes):
    super().__init__()
    self.conv1=Conv2D(32,(3,3),activation='relu',input_shape = (width ,height ,3))
    self.conv2=Conv2D(64,(3,3),activation='relu')
    self.conv3=Conv2D(128,(3,3),activation='relu')
    self.conv4=Conv2D(256,(3,3),activation='relu')
    self.conv5=Conv2D(512,(3,3),activation='relu')
    self.pooling1=MaxPooling2D((3,3))
    self.pooling2=AveragePooling2D((2,2))
    self.flatten=Flatten()
    self.fc0=Dense(1024,activation='relu')
    self.fc1=Dense(512,activation='relu')
    self.fc2=Dense(256,activation='relu')
    self.fc3=Dense(128,activation='relu')
    self.fc3=Dense(64,activation='relu')
    self.fc4=Dense(number_of_classes,activation='softmax')
  
  def call(self,x):
    y=self.conv1(x)
    f=self.pooling1(y)
    y=self.conv2(f)
    f=self.pooling1(y)
    y=self.conv3(f)
    # f=self.pooling2(y)
    y=self.conv4(y)
    y=self.conv5(y)

    w=self.flatten(y)
    z=self.fc0(w)
    z=self.fc1(z)
    z=self.fc2(z)
    z=self.fc3(z)
    out=self.fc4(z)
    return out

model=MyModel(14)


In [13]:
loss_function=tf.keras.losses.CategoricalCrossentropy(from_logits=True)
optimizer=tf.keras.optimizers.Adam()

In [14]:
train_loss=tf.keras.metrics.Mean(name='train_loss')
train_acc=tf.keras.metrics.CategoricalAccuracy(name='train_acc')

test_loss=tf.keras.metrics.Mean(name='test_loss')
test_acc=tf.keras.metrics.CategoricalAccuracy(name='test_acc')

In [15]:
def train_step(images,y):
  with tf.GradientTape() as tape:
    y_pred=model(images,training=True)
    loss=loss_function(y,y_pred)

  gradients=tape.gradient(loss,model.trainable_variables)
  optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  train_loss(loss)
  train_acc(y,y_pred)

In [16]:
def test_step(images,y):
  y_pred=model(images,training=False)
  loss=loss_function(y,y_pred)
  test_loss(loss)
  test_acc(y,y_pred)

In [17]:
from tqdm import tqdm
def train():
  epochs=30
  tSteps=train_data.samples//batch_size
  vSteps=val_data.samples//batch_size
  for epoch in range(epochs):
    train_loss.reset_states()
    train_acc.reset_states()
    test_loss.reset_states()
    test_acc.reset_states()
    for i in tqdm(range(tSteps)):
        images, labels = next(train_data)
        train_step(images, labels)

    for i in tqdm(range(vSteps)):
        images, labels = next(val_data)
        test_step(images, labels)
    print('epoch: ',epoch+1)
    print('loss: ',train_loss.result())
    print('accuracy: ',train_acc.result())
    print('val loss: ',test_loss.result())
    print('val accuracy: ',test_acc.result())

In [18]:
train()

  0%|          | 0/34 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/keras/backend.py:4847: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`categorical_crossentropy` received `from_logits=True`, but '
100%|██████████| 8/8 [00:04<00:00,  1.86it/s]


epoch:  1
loss:  tf.Tensor(2.7073302, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.07176582, shape=(), dtype=float32)
val loss:  tf.Tensor(2.6359527, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.11440678, shape=(), dtype=float32)


100%|██████████| 8/8 [00:04<00:00,  1.88it/s]


epoch:  2
loss:  tf.Tensor(2.5775661, shape=(), dtype=float32)
accuracy:  tf.Tensor(0.12181303, shape=(), dtype=float32)
val loss:  tf.Tensor(2.43827, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.13983051, shape=(), dtype=float32)


100%|██████████| 8/8 [00:04<00:00,  1.86it/s]


...

epoch:  28
loss:  tf.Tensor(0.00014471982, shape=(), dtype=float32)
accuracy:  tf.Tensor(1.0, shape=(), dtype=float32)
val loss:  tf.Tensor(1.7994089, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.7923729, shape=(), dtype=float32)


100%|██████████| 8/8 [00:04<00:00,  1.88it/s]


epoch:  29
loss:  tf.Tensor(4.826682e-05, shape=(), dtype=float32)
accuracy:  tf.Tensor(1.0, shape=(), dtype=float32)
val loss:  tf.Tensor(1.6397547, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.7966102, shape=(), dtype=float32)


100%|██████████| 8/8 [00:04<00:00,  1.89it/s]

epoch:  30
loss:  tf.Tensor(3.588354e-05, shape=(), dtype=float32)
accuracy:  tf.Tensor(1.0, shape=(), dtype=float32)
val loss:  tf.Tensor(1.7968082, shape=(), dtype=float32)
val accuracy:  tf.Tensor(0.7881356, shape=(), dtype=float32)


In [17]:
model.save_weights(filepath='checkpoint',save_format='HDF5')
# model.save('faces.h5')